In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

path = r'C:\chromedriver-win64\chromedriver.exe'
service = Service(path)
service.start()

driver = webdriver.Chrome(path, options=options)
driver.get("https://www.psx.com.pk/market-summary/")


data = []

rows = driver.find_elements(By.XPATH, "//table//tbody/tr[substring(@class, string-length(@class) - string-length('text-td') + 1) = 'text-td']")

# print(rows)
for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    data.append({
        "SCRIP": cols[0].text if len(cols) > 0 else '',
        "LDCP": cols[1].text if len(cols) > 1 else '',
        "OPEN": cols[2].text if len(cols) > 2 else '',
        "HIGH": cols[3].text if len(cols) > 3 else '',
        "LOW": cols[4].text if len(cols) > 4 else '',
        "CURRENT": cols[5].text if len(cols) > 5 else '',
        "CHANGE": cols[6].text if len(cols) > 6 else '',
        "Volume": cols[7].text if len(cols) > 7 else ''
    })

driver.quit()

df = pd.DataFrame(data)
df.to_excel("psx_market_summary.xlsx", index=True)

print("✅ Data saved to 'psx_market_summary.xlsx'")


✅ Data saved to 'psx_market_summary.xlsx'


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the Excel data
df = pd.read_excel("psx_market_summary.xlsx", engine='openpyxl')

# Clean and convert numeric columns
numeric_cols = ["LDCP", "OPEN", "HIGH", "LOW", "CURRENT", "CHANGE", "Volume"]
for col in numeric_cols:
    df[col] = df[col].replace({',': '', '--': None}, regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing essential data
df.dropna(subset=["CURRENT", "Volume"], inplace=True)

# Get Top 10 most traded stocks by volume
top10 = df.sort_values(by="Volume", ascending=False).head(10)

# Interactive Bar Chart: Current Price
fig1 = px.bar(
    top10,
    x='SCRIP',
    y='CURRENT',
    color='CHANGE',
    text='CURRENT',
    color_continuous_scale='Tealgrn',
    title='Top 10 PSX Stocks by Volume - Current Price',
    labels={'CURRENT': 'Current Price', 'SCRIP': 'Stock'}
)
fig1.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig1.update_layout(
    yaxis_title='Current Price (PKR)',
    xaxis_title='Stock Symbol',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    template='plotly_white',
    title_x=0.5
)

# Line Chart: High, Low, and Current of Top 5
top5 = top10.head(5)
fig2 = go.Figure()

for metric in ['HIGH', 'LOW', 'CURRENT']:
    fig2.add_trace(go.Scatter(
        x=top5['SCRIP'],
        y=top5[metric],
        mode='lines+markers',
        name=metric,
        line=dict(width=2)
    ))

fig2.update_layout(
    title="Price Trend: High, Low, and Current (Top 5 Stocks)",
    xaxis_title="Stock Symbol",
    yaxis_title="Price (PKR)",
    template='plotly_white',
    title_x=0.5,
    legend_title="Price Type"
)

# Show plots
fig1.show()
fig2.show()
